In [4]:
#PROJECT-1  Live Cricket Score

!pip install requests beautifulsoup4 lxml pandas

import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from datetime import datetime
from google.colab import files

uploaded = files.upload()
dataset_filename = list(uploaded.keys())[0]
df_dataset = pd.read_csv(dataset_filename)

print("📂 Uploaded Dataset (first 5 rows):")
display(df_dataset.head())

url = "https://www.espncricinfo.com/rss/livescores.xml"
response = requests.get(url)
response.raise_for_status()
soup = BeautifulSoup(response.content, "xml")
items = soup.find_all("item")

matches = []
pattern = r"(.+?)\s+(\d+/\d+)\s+\(([\d\.]+)\)\s+vs\s+(.+?)\s+(\d+/\d+)\s+\(([\d\.]+)\)"

for it in items:
    title = it.title.text.strip()
    desc = it.description.text.strip()

    match_data = {
        "Match": title,
        "Team 1": "", "Score 1": "", "Overs 1": "",
        "Team 2": "", "Score 2": "", "Overs 2": ""
    }

    m = re.search(pattern, desc)
    if m:
        match_data.update({
            "Team 1": m.group(1).strip(),
            "Score 1": m.group(2).strip(),
            "Overs 1": m.group(3).strip(),
            "Team 2": m.group(4).strip(),
            "Score 2": m.group(5).strip(),
            "Overs 2": m.group(6).strip(),
        })
    else:
        match_data["Overs 2"] = desc

    matches.append(match_data)

df_live = pd.DataFrame(matches)



if "Team" in df_dataset.columns:
    team_list = df_dataset["Team"].dropna().unique().tolist()
elif "Teams" in df_dataset.columns:
    team_list = df_dataset["Teams"].dropna().unique().tolist()
else:
    team_list = []

df_filtered = df_live[
    (df_live["Team 1"].isin(team_list)) | (df_live["Team 2"].isin(team_list))
]

print("📊 Filtered Live Scores (based on uploaded dataset):")
display(df_filtered)

filename_filtered = f"filtered_live_scores_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
df_filtered.to_csv(filename_filtered, index=False)

files.download(filename_filtered)
print(f" Filtered live scores saved to {filename_filtered}")


Saving t20I_cricket_dataset.csv to t20I_cricket_dataset (2).csv
📂 Uploaded Dataset (first 5 rows):


,Unnamed: 0,powerPlay,AverageScore,battingTeam,bowlingTeam,city,delivery_left,score,CurrentRunRate,wicketsLeft,Run_In_Last5,Wickets_In_Last5,Final_Score,innings
0,10400,0,138.162791,New Zealand,Pakistan,Dubai,63,56,5.894737,8,24.0,2.0,144,1
1,163545,0,103.000000,Zimbabwe,Sri Lanka,Hambantota,40,76,5.700000,5,30.0,2.0,100,2
2,1557,0,154.000000,West Indies,Australia,St Lucia,53,68,6.089552,5,27.0,3.0,105,1
3,94589,0,146.285714,Australia,India,Bengaluru,18,159,9.352941,7,50.0,0.0,194,2
4,119086,0,135.609756,Sri Lanka,South Africa,Colombo,42,82,6.307692,7,38.0,1.0,135,2


📊 Filtered Live Scores (based on uploaded dataset):


,Match,Team 1,Score 1,Overs 1,Team 2,Score 2,Overs 2


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

 Filtered live scores saved to filtered_live_scores_20250925_145205.csv
